In [1]:
## The DSA needs to have appropriate metadata in order to view a multiplex image appropriately in the faceted search viewer
import girder_client
import gcSecrets
import tqdm
%load_ext autoreload
%autoreload 2
import colorsys

gc= girder_client.GirderClient(apiUrl=gcSecrets.styxApiUrl)
gc.authenticate(apiKey=gcSecrets.styxApiToken)

{'_id': '5cacfa86e62914004a60e139'}

In [2]:
vandyMxIfTestFolder = "6390b2744da1ec8c4ffd11f9"

mxifImageSet = list(gc.listItem(vandyMxIfTestFolder))
print(len(mxifImageSet))

6


In [25]:
channelTemplates = [{"template": "baseMarkers", "markers": ["DAPI","PANCK","VIMENTIN","COLLAGEN","CD45"]},
                    {"template": "Segmentation","markers": ["DAPI","NAKATPASE","VIMENTIN","SMA","COLLAGEN"]},
                    {"template": "Immune", "markers": ["DAPI","CD45","CD3","CD68","CD20"]},
                    {"template": "Epithelium","markers": ["DAPI","BCATENIN","PANCK","NAKATPASE","PEGFR"]},
                    {"template": "Adenoma", "markers": ["DAPI","MUC5AC","CDX2","OLFM4","BCATENIN"]},
                    {"template": "Immune Exclusion", "markers": ["DPEP1","DDR1","TGFBI","CD8"]} 
                   ]

rgbColorMap = [(51, 255, 255),
            (255,0,0),
            (0,255,0),
            (255,170,51),
            (170,51,255)]


rgbColorMap = ['rgb(51, 255, 255)',
               'rgb(255,0,0)',
               'rgb(0,255,0)',
               'rgb(255,170,51)',
                'rgb(170,51,255)']

# hslColorMap = []
# for c in rgbColorMap:
#     hslColorMap.append(  colorsys.rgb_to_hls(*c))

def generateMxIFpresets( item ):
    ### This will accept an item from the DSA and try and see if the auto color templates exist
    if 'DSAGroupSet' in item['meta']:
        DSAGroupSet = item['meta']['DSAGroupSet']
        ## Thing I probably want here is the name
    else:
        DSAGroupSet = []
        
    if 'ioparams' in item['meta']:
        channelmap = item['meta']['ioparams']['channelmap']
    else:
        print("No channel map detected..")
        return
    channelmapUpper = {k.upper():v for k,v in channelmap.items()}
        
    for g in DSAGroupSet:
        print(g['name'])

    curGroups = [x['name'] for x in DSAGroupSet]
    
    for t in channelTemplates:
        ### Try to see if each channel template already exists or if I need to create it..
        #print(t['template'])
        
        curTemplate = {'channels':[], 'name':t['template']}
        for i,m in enumerate(t['markers']):
            if m in channelmapUpper:
                #print("Found",m,"at frame",channelmapUpper[m])
                curTemplate['channels'].append({'max':15000, 'min':500,'opacity':1, 
                                               'name':m,'color':rgbColorMap[i],
                                               'index':channelmapUpper[m]})
        ### Need to add a force update flag once this works..
        if t['template'] in curGroups:
            print("Template is already in the DSA Group Set")
        else:
            DSAGroupSet.append(curTemplate)
                 
    ## Need to get the names of the current sets..    
    ## kind of annoying, but I have to see if the template already exists, and then overwrite it if that flag is set
    return DSAGroupSet    

In [26]:
for img in mxifImageSet:
    dsag = generateMxIFpresets(img)
    gc.addMetadataToItem(img['_id'],{'DSAGroupSet':dsag})
    print([x['name'] for x in dsag])


CD45_CD4_CD68_CD8_CgA
baseMarkers
baseMarkers
baseMarkers
Segmentation
Immune
Epithelium
Template is already in the DSA Group Set
Template is already in the DSA Group Set
Template is already in the DSA Group Set
Template is already in the DSA Group Set
['CD45_CD4_CD68_CD8_CgA', 'baseMarkers', 'baseMarkers', 'baseMarkers', 'Segmentation', 'Immune', 'Epithelium', 'Adenoma', 'Immune Exclusion']
['baseMarkers', 'Segmentation', 'Immune', 'Epithelium', 'Adenoma', 'Immune Exclusion']
DAPI_PanCK_Vimentin_Collagen_CD45
baseMarkers
baseMarkers
Segmentation
Immune
Epithelium
Template is already in the DSA Group Set
Template is already in the DSA Group Set
Template is already in the DSA Group Set
Template is already in the DSA Group Set
['DAPI_PanCK_Vimentin_Collagen_CD45', 'baseMarkers', 'baseMarkers', 'Segmentation', 'Immune', 'Epithelium', 'Adenoma', 'Immune Exclusion']
DAPI_CD45_Collagen_Vimentin_PanCK
baseMarkers
baseMarkers
Segmentation
Immune
Epithelium
Template is already in the DSA Group 